In [1]:
import pickle
import numpy as np
import scipy as sp
import gym
import gym_atena.envs.atena_env_cont as atena_env_cont
from gym_atena.envs.atena_env_cont import ATENAEnvCont
import Configuration.config as cfg
from Evaluation.notebook_utils import *

%matplotlib inline


Gym has been unmaintained since 2022 and does not support NumPy 2.0 amongst other critical functionality.
Please upgrade to Gymnasium, the maintained drop-in replacement of Gym, or contact the authors of your software and request that they upgrade.
See the migration guide at https://gymnasium.farama.org/introduction/migration_guide/ for additional information.


Registering ATENAcont-v0 environment
Configuration loaded with:
  - humanity_coeff: 1.0
  - diversity_coeff: 2.0
  - kl_coeff: 1.5
  - compaction_coeff: 2.0
  - adam_lr: 0.0003
  - ppo_gamma: 0.995
  - ppo_lambda: 0.97


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/pytz/tzinfo.py:27: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  _epoch = datetime.utcfromtimestamp(0)


✅ Using Snorkel compatibility adapter




































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/pandas/core/algorithms.py:522: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
  common = np.find_common_type([values.dtype, comps_array.dtype], [])
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/pandas/core/algorithms.py:522: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
  common = np.find_common_type([values.dtype, comps_array.dtype], [])
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/pandas/core/algorithms.py:522: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_typ

# Load Human Sessions


In [5]:
# Fix Environment Wrapping Issue - Unwrap to access ATENAEnvCont directly
env = env.unwrapped
print(" Environment unwrapped successfully")


✅ Environment unwrapped successfully


In [2]:
%%capture
repo=Repository('gym_atena/reactida/session_repositories/actions_atena.tsv','gym_atena/reactida/session_repositories/displays.tsv','gym_atena/reactida/raw_datasets/')

# Filter Solutions Only
solutions_df, solutions_ids = get_solution_sessions_dfs_and_id(repo)

# create env
env = gym.make(env_d)
env.reset()

# Reset collisions set
observation_collisions = set()


# Validate Successful Loading


In [3]:
repo.actions.head(10)


,action_id,action_type,action_params,session_id,user_id,project_id,creation_time,parent_display_id,child_display_id,solution
0,1,group,"{'field': 'eth_src', 'aggregations': [], 'grou...",1,1,1,2016-08-14 12:44:05,1,2,True
1,2,group,"{'field': 'ip_src', 'aggregations': [], 'group...",1,1,1,2016-08-14 12:44:08,2,3,True
2,3,group,"{'field': 'eth_src', 'aggregations': [{'field'...",2,5,1,2016-08-15 09:40:42,4,5,True
3,0,back,{},2,5,1,,5,4,True
4,4,group,"{'field': 'ip_src', 'aggregations': [{'field':...",2,5,1,2016-08-15 13:13:54,4,6,True
5,0,back,{},2,5,1,,6,4,True
6,5,group,"{'field': 'ip_src', 'aggregations': [], 'group...",2,5,1,2016-08-15 13:14:10,4,7,True
7,6,filter,"{'field': 'info_line', 'term': 'reply', 'condi...",2,5,1,2016-08-15 13:15:13,7,8,True
8,0,back,{},2,5,1,,8,7,True
9,8,filter,"{'field': 'highest_layer', 'term': 'ICMP', 'co...",2,5,1,2016-08-15 13:16:34,7,10,True


# Cluster Human Sessions By Observation Vector


In [6]:
# key is the hashed value of the current DataFrame object
# value is a a dict where key is a HumanStep namedtuple and key is
# a tuple of current state (to reproduce the current DataFrame) and
# a dataset_number
human_displays_actions_clusters = defaultdict(dict)

# key is the current observation (a tuple of length obsevation vector)
# value is a list of action vectors taken for this observation
human_displays_actions_clusters_obs_based = defaultdict(list)

# solution ids with illegal filters or time_sniff
bad_sol_ids = [57, 622]

# DataFrames that there hashes collide
collision_hashes = [
'39e5a8ef59a77463fd0176bbd800f95e7d63460f2f461323597b1e38e177bdca',
'e3b0c44298fc1c149afbf4c8996fb92427ae41e4649b934ca495991b7852b855',
'8d591772a43d763b9377e40f9d5fa5ae903cfd7572f6c51eec527ceb51decdc0',
'777c2dfd624aa21d74216d9263d198ce92359d353eaf51531e4aff1707fa2ade']

# Maximum Sessions to Run
max_sessions_to_run = 1000

# cluster all sessions
for idx, sol_id in enumerate(solutions_ids):
    if sol_id not in bad_sol_ids:
        print("\n-------------------------------------------------------------------\n")
        print("Clustering session number %d with sol_id number %d" % (idx, sol_id))
        add_human_session_to_clusters(env, sol_id, solutions_df,
                                  human_displays_actions_clusters,
                                  human_displays_actions_clusters_obs_based,
                                  observation_collisions,)
    if idx >= max_sessions_to_run:
        break



-------------------------------------------------------------------

Clustering session number 0 with sol_id number 1
Dataset id is 0
{'field': 'eth_src', 'aggregations': [], 'groupPriority': 0}
{'field': 'ip_src', 'aggregations': [], 'groupPriority': 1}


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/gym/core.py:49: DeprecationWarning: WARN: You are calling render method, but you didn't specified the argument render_mode at environment initialization. To maintain backward compatibility, the environment will render in human mode.
If you want to render in human mode, initialize the environment in this way: gym.make('EnvName', render_mode='human') and don't call the render method.
See here for more information: https://www.gymlibrary.ml/content/api/
  deprecation(


🔧 Rule-based humanity coefficient applied: 1.0
🎯 Using REAL LabelModel.predict_proba() on (1, 51) matrix
✅ Real Snorkel predictions: mean=0.000, std=0.000
🔧 Diversity coefficient applied (main calc): 2.0
🔧 Compaction coefficient applied: 2.0
🎯 MASTER-EXACT REWARD:
   action_type: 2
   kl_distance: 0
   compaction_gain: 1.9823869540707095
   total_interestingness: 1.9823869540707095
🎯 Using REAL LabelModel.predict_proba() on (1, 51) matrix
✅ Real Snorkel predictions: mean=0.000, std=0.000
🔧 BEFORE coeff: r_snorkel_humanity=-0.999931
🔧 Using current humanity_coeff: 1.0
🔧 AFTER coeff: r_snorkel_humanity=-0.999931
🎯 Snorkel humanity: -0.999931
[2 2 0 0 0 0]


"Group on Column 'eth_src' and aggregate with '<built-in function len>' on the column 'packet_number'"

🔧 Rule-based humanity coefficient applied: 1.0
🎯 Using REAL LabelModel.predict_proba() on (1, 51) matrix
✅ Real Snorkel predictions: mean=0.000, std=0.000
🔧 Diversity coefficient applied (main calc): 2.0
🔧 Compaction coefficient applied: 2.0
🎯 MASTER-EXACT REWARD:
   action_type: 2
   kl_distance: 0
   compaction_gain: 0.14675573402404612
   total_interestingness: 0.14675573402404612
🎯 Using REAL LabelModel.predict_proba() on (1, 51) matrix
✅ Real Snorkel predictions: mean=0.000, std=0.000
🔧 BEFORE coeff: r_snorkel_humanity=-0.999970
🔧 Using current humanity_coeff: 1.0
🔧 AFTER coeff: r_snorkel_humanity=-0.999970
🎯 Snorkel humanity: -0.999970
[2 6 0 0 0 0]


...

"Group on Column 'ip_src' and aggregate with '<built-in function len>' on the column 'packet_number'"

🎯 Using REAL LabelModel.predict_proba() on (1, 51) matrix
✅ Real Snorkel predictions: mean=0.000, std=0.000
🔄 BACK ACTION: humanity rewards only
🎯 Using REAL LabelModel.predict_proba() on (1, 51) matrix
✅ Real Snorkel predictions: mean=0.000, std=0.000
🔧 BEFORE coeff: r_snorkel_humanity=-0.999770
🔧 Using current humanity_coeff: 1.0
🔧 AFTER coeff: r_snorkel_humanity=-0.999770
🎯 Snorkel humanity: -0.999770
[0 0 0 0 0 0]


'Back'

🔧 Rule-based humanity coefficient applied: 1.0
🎯 Using REAL LabelModel.predict_proba() on (1, 51) matrix
✅ Real Snorkel predictions: mean=0.000, std=0.000
🔧 Diversity coefficient applied (main calc): 2.0
🔧 Compaction coefficient applied: 2.0
🎯 MASTER-EXACT REWARD:
   action_type: 2
   kl_distance: 0
   compaction_gain: 1.9768998580378292
   total_interestingness: 1.9768998580378292
🎯 Using REAL LabelModel.predict_proba() on (1, 51) matrix
✅ Real Snorkel predictions: mean=0.000, std=0.000
🔧 BEFORE coeff: r_snorkel_humanity=-0.999796
🔧 Using current humanity_coeff: 1.0
🔧 AFTER coeff: r_snorkel_humanity=-0.999796
🎯 Snorkel humanity: -0.999796
[2 5 0 0 0 0]


"Group on Column 'ip_dst' and aggregate with '<built-in function len>' on the column 'packet_number'"

# Validate Clusters Creation


In [7]:
print("Number of clusters: %d" % len(human_displays_actions_clusters_obs_based))
print("Max number of action for a single observation: %d" % max([len(list(human_displays_actions_clusters_obs_based.values())[i]) for i in range(len(human_displays_actions_clusters_obs_based))]))
print("\n\n")
for idx, (key, vals) in enumerate(human_displays_actions_clusters_obs_based.items()):
    print("Observation:\n" + str(key))
    print()
    print("Actions:\n" + str(vals))
    break
    print("\n\n")


Number of clusters: 197
Max number of action for a single observation: 104



Observation:
(1.0, 0.0, 1.0, 0.00023126735, 0.0, 0.03441567, 0.00023126735, 0.0, 0.03441567, 0.000346901, 0.0, 0.039079428, 0.94229877, 0.0, 0.98828954, 0.42703515, 0.0, 0.850694, 0.020929694, 0.0, 0.08709805, 0.0011563367, 0.0, 0.16330612, 0.0136447735, 0.0, 0.523395, 0.01827012, 0.0, 0.07943899, 0.024167437, 0.0, 0.08619737, 0.023936171, 0.0, 0.10307587, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, 0.0, 0.0, 0.0)

Actions:
[array([2, 2, 0, 0, 0, 0]), array([2, 2, 0, 0, 0, 0]), array([2, 6, 0, 0, 0, 0]), array([2, 6, 0, 0, 0, 0]), array([ 1,  5,  0, 56,  0,  0]), array([2, 6, 0, 0, 0, 0]), array([2, 6, 0, 0, 0, 0]), array([2, 2, 0, 0, 0, 0]), array([ 1, 11,  0, 81,  0,  0]), array([  1,   5,   0, 112,   0,   0]), array([2, 3, 0, 0, 0, 0]), array([2, 3, 0, 0, 0, 0]), array([2, 2, 0, 0, 0, 0]), array([ 1,  4,  6, 13,  0,  0]), array([2, 3, 0, 0, 0, 0]), array([ 1,  3,  5, 50,  0,  0]

# Save Cluster to .pickle file


In [8]:
# Set to True if you would like to rewrite the existing .pickle file (otherwise this cell do nothing)
WRITE_TO_PKL = False

if WRITE_TO_PKL:
    # Store data (serialize)
    with open('human_actions_clusters.pickle', 'wb') as handle:
        pickle.dump(human_displays_actions_clusters_obs_based, handle, protocol=pickle.HIGHEST_PROTOCOL)
